In [ ]:
!pip install galileo-observe galileo-protect openai

In [ ]:
import galileo_protect as gp
from openai import AzureOpenAI
import galileo_observe as observe
import datetime, os, json

Azure OpenAI Stuff

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"]="" # Your API Key from Azure

In [ ]:
client = AzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint = ""# Your deployment endpoint
    )

Galileo Stuff

In [ ]:
os.environ["GALILEO_CONSOLE_URL"]="" # The URL to your console
os.environ["GALILEO_API_KEY"]="" # Bottom Left Icon > Settings > API Keys

In [ ]:
project = gp.create_project('bot_protect_example')
project_id = project.id

observe_logger = observe.ObserveWorkflows(project_name="bot_observe_example")

stage = gp.create_stage(name="first_stage", project_id=project_id)
stage_id = stage.id

In [ ]:
# Payload from Galileo Protect
def protect_payload(input: str, output: str):
  return gp.Payload(
      input=input,
      output=output
  )

In [ ]:
# Function from Galileo Protect
def protect_galileo(input: str, output: str):
    protect_out = gp.invoke(
        payload=protect_payload(input, output),
        prioritized_rulesets=[
            {
                "rules": [
                    {
                        "metric": "pii",
                        "operator": "contains",
                        "target_value": "ssn",
                    },
                ],
                "action": {
                    "type": "OVERRIDE",
                    "choices": [
                        "Personal Identifiable Information detected in the model output. Sorry, I cannot answer that question."
                    ],
                },
            },
        ],
        stage_id=stage_id,
        timeout=3,
    )
    return protect_out

In [ ]:
# This is the function that we want the model to be able to call
def get_delivery_date(order_id: str) -> str:
    return datetime.datetime.now()

In [ ]:
# Define the tools you would like to use
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    }
]

In [ ]:
# Conversational Flow
inputs = [
{"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
{"role": "user", "content": "Hi, can you tell me the delivery date for my order?"},
{"role": "user", "content": "My order number is 12345."}
]

In [ ]:
for i, prompt in enumerate(inputs):
    if prompt["role"] == "system":
        wf = observe_logger.add_workflow(input={"input": prompt["content"]}, output={"output": str(tools)}, name="CustomWorkflow")

    if prompt["role"] == "user":
        current_messages = inputs[:i+1]
        # Conduct chat completion
        response = client.chat.completions.create(
          model="gpt-35-turbo",
          messages=current_messages,
          tools=tools,
        )
        output_message = response.choices[0].message

        # Check if it is an LLM or a Tool message
        if output_message.content is not None:
            wf.add_llm(
            input=prompt["content"],
            output=output_message.content,
            model="gpt-35-turbo",
            input_tokens=response.usage.prompt_tokens,
            output_tokens=response.usage.completion_tokens,
            total_tokens=response.usage.total_tokens,
            metadata={"Node": "LLM"}
            )
        elif output_message.tool_calls is not None:
            order_id = json.loads(response.choices[0].message.tool_calls[0].function.arguments)["order_id"]
            tool_output = get_delivery_date(order_id)
            wf.add_tool(
            input=response.choices[0].message.tool_calls[0].function.arguments,
            output=f"Your delivery date is {tool_output}.",
            name=response.choices[0].message.tool_calls[0].function.name,
            metadata={"Node": "Function"}
            )

        # On LLM steps, also run Protect
        if output_message.content is not None:
          response_protect = protect_galileo(prompt["content"], output_message.content)

          wf.add_protect(
              payload=protect_payload(prompt["content"], output_message.content),
              response=response_protect,
              metadata={"Node": "Protect"}
          )

    # Conclude the workflow
    wf.conclude(output={"output": output_message.content})

# Upload all logged workflows to Galileo after processing
observe_logger.upload_workflows()